On accumule tous des pièces de monnaies. Soit parce qu’on oublie qu’ont les as et on ne se rend même pas compte, soit parce que nous ne sommes pas bons en calcul mental et nous n'avons pas envie de conter toutes les pièces et on préfère payer avec des billets ou par carte parce que c’est plus rapide. Quoi qu’il en soit on finit toujours par rassembler une plus ou moins importante quantité de pièces dans une tirelire ou dans une caisse sous notre lit ou caché dans un tiroir. M'aimerais tu pas savoir combien d'argent tu as en pièce? Grâce à ce programme aujourd'hui c'est possible! Juste prend une photo de tes pièces et hop le programme te donne la valeur totale de toutes tes pièces.

La première chose à faire est d'importer toutes les librairies externes qu'on va utiliser dans le programme

In [1]:
import os 
import cv2 # Computer vision -- traitement d'image
import numpy as np # array --> matrix de plus de deux dimention
import sys
import tkinter.filedialog

Pour qu'une intelligence artificielle puisse reconnaîte quelque chose il faut d'abord lui montrer des exemples de ce qu'il va devoir reconnaître, ceci s'appele l'entrainement. Pour faire cela ont d'abord téléchargé 1291 photos de pièces d'euros qu'on a séparé en différents fichiers correspondant à la valeur des pièces

In [2]:
datadir_train = "classified/train"
datadir_test = "classified/test"
categories = ['1c', '2c', '5c', '10c', '20c', '50c', '1e', '2e']

Il y a deux façons pour un ordinateur de reconnaître une pièce, soit par les dimensions de la pièce (ce qui peut être très précis) ou en analysant toutes les différentes variations de couleurs. Pour avoir les dimensions il faut une échelle et malheureusement les photos qu'on à téléchargées n'avaient d'échelle, de plus ce n'est pas pratique pour l'utilisateur de devoir rajouter une échelle aux photos qu'il veut analyser. 

Pour pouvoir analyser les couleurs des photos on a donc fait une boucle qui passe par toutes les photos et qui rajoute une liste des valeurs RGB de chaque photo dans une autre liste : training_data

In [5]:
training_data = []
for category in categories:
    path = os.path.join(datadir_train, category) # path est la location de la photo
    label = categories.index(category) # les photos sont séparé en catégorie donc à chaque photo on associe un nombre dépandant de la catégorie dans la quel se trouve la photo
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img))
        training_data.append([img_array, label]) # on rajoute les liste de valeurs RGB à la liste training_data

On a fait la même chose avec 80 photos qu'on va utiliser pour tester la précision de notre programme 

In [7]:
testing_data = []
for category in categories:
    path = os.path.join(datadir_test, category)
    class_num = categories.index(category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img))
        testing_data.append([img_array, class_num])

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'classified/test\\1c'

Ici on sépare les valeurs RGB et les 

In [ ]:
x_train = []
y_train = []

for features, label in training_data:
    x_train.append(features)
    y_train.append(label)
    
x_train = np.array(x_train)

In [ ]:
x_test = []
y_test = []

for features, label in testing_data:
    x_test.append(features)
    y_test.append(label)
    
x_test = np.array(x_test)

In [274]:
from collections import Counter
def k_nearest_neighbors(predict, k):
    distences = []
    for image in training_data:
        distences.append([np.linalg.norm(image[0] - predict), image[1]])
    distences.sort()
    votes = [i[1] for i in distences[:k]]
    votes = ''.join(str(e) for e in votes)
    votes = votes.replace(',', '')
    votes = votes.replace(' ', '')
    result = Counter(votes).most_common(1)[0][0]
    return result

In [275]:
y_test = ''.join(str(e) for e in y_test)

In [286]:
start = time()
correct = 0
total = 0
for image in testing_data:
    prediction = k_nearest_neighbors(image[0], 1)
    if int(prediction) == image[1]:
        correct += 1
    total += 1
accuracy = correct/total
end = time()
print(end-start)

1.0132901668548584


In [287]:
print(accuracy)

0.7


In [1]:
filename = tkinter.filedialog.askopenfilename(title="Ouvrir fichier", filetypes=[('all files', '.*')])
# Loads an image
src = cv2.imread(cv.samples.findFile(filename), cv.IMREAD_COLOR)

In [ ]:
def resize_img(img):
    scale_percent = 500 / img.shape[1]
    w = int(img.shape[1] * scale_percent)
    h = int(img.shape[0] * scale_percent)
    dim = (w, h)
    new_img = cv.resize(img, dim)
    return new_img

In [ ]:
new_img = resize_img(src)

gray = cv2.cv2tColor(new_img, cv2.COLOR_BGR2GRAY)

gray = cv2.medianBlur(gray, 5)

rows = gray.shape[0]
circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, rows / 8, param1=100, param2=30, minRadius=1, maxRadius=70)

circle_img = new_img
numero_piece = 0
Images = []
if circles is not None:
    circles = np.uint16(np.around(circles))
    for i in circles[0,:]:

        radius = i[2]

        img_name = str('piece' + str(numero_piece))
        img_cropped = new_img[i[1] - (radius + 5):i[1] + (radius + 5), i[0] - (radius + 5):i[0] + (radius + 5), ]
        numero_piece += 1
        Images.append(img_cropped)
        #cv2.imshow(img_name, img_cropped)
        #cv2.imwrite(img_name + '.jpg', img_cropped)

cv2.imshow("detected circles", circle_img)

cv2.waitKey(0)

In [268]:
def count_money(prediction):
    somme_de_pieces = 0
    for image in Images:
        prediction = k_nearest_neighbors(image[0], 30)
        if prediction == 0:
            somme_de_pieces += 0.01
        if prediction == 1:
            somme_de_pieces += 0.02
        if prediction == 2:
            somme_de_pieces += 0.05
        if prediction == 3:
            somme_de_pieces += 0.10
        if prediction == 4:
            somme_de_pieces += 0.20
        if prediction == 5:
            somme_de_pieces += 0.50
        if prediction == 6:
            somme_de_pieces += 1.00
        if prediction == 7:
            somme_de_pieces += 2.00

SyntaxError: unexpected EOF while parsing (<ipython-input-268-2c607de7a7be>, line 4)